# Statistics

In [1]:
%load_ext autoreload
%autoreload 2

## Graph statistics

In [ ]:
import graph_helper
import dataset_helper
import matplotlib.pyplot as plt
from IPython.display import display

for graph_cache_file in dataset_helper.get_all_cached_graph_datasets():
    if 'ana' in graph_cache_file: continue
    if 'gml' not in graph_cache_file: continue
    cache_filename = graph_cache_file.split('/')[-1]
    print('{} {}'.format('#' * 20, cache_filename))
    X, Y = dataset_helper.get_dataset_cached(graph_cache_file)
    num_classes = min(len(set(X)), 20)
    #dataset_helper.plot_dataset_class_distribution(X, Y, title = graph_cache_file, figsize=(12, num_classes * 0.8))
    df = graph_helper.get_graph_stats(X, Y)
    
    ax = df['num_graphs'].plot.barh(title = 'Graphs per topic\n(Dataset: {})'.format(cache_filename.replace('.npy', '')), legend = True, figsize = (14, 8))
    ax.set_xlabel('# graphs')
    display(df)
    plt.show()
    break

In [12]:
from glob import glob 
import pickle
import pandas as pd
from IPython.display import display

df_all = None
for result_file in glob('data/results/*.npy'):
    with open(result_file, 'rb') as f:
        result = pickle.load(f)
    result_df = pd.DataFrame(result).sort_values(by = 'mean_test_score', ascending = False)
    result_df['dataset'] = result_file.split('/')[-1].rsplit('.', 4)[0]
    if df_all is None:
        df_all = result_df
    else:
        df_all = df_all.append(result_df)
df_all.sort_val

,dataset,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_clf,param_clf__class_weight,param_clf__max_iter,param_clf__tol,param_count_vectorizer__stop_words,...,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
1,dataset_graph_cooccurrence_1_all_cade-ana,569.889517,90.176428,0.522354,0.831375,"PassiveAggressiveClassifier(C=1.0, average=Fal...",balanced,1000,0.001,NaN,...,0.523372,0.838540,0.524048,0.818447,0.519640,0.837136,368.673860,2.508121,0.001938,0.009159
0,dataset_graph_cooccurrence_1_all_cade-ana,1070.279949,89.832656,0.458226,0.911385,"PassiveAggressiveClassifier(C=1.0, average=Fal...",balanced,1000,0.001,NaN,...,0.461116,0.921792,0.463896,0.925551,0.449662,0.886812,8.557186,7.918151,0.006160,0.017443
3,dataset_graph_cooccurrence_1_all_cade-ana,321.561983,93.182603,0.280987,0.861869,"PassiveAggressiveClassifier(C=1.0, average=Fal...",balanced,1000,0.001,NaN,...,0.290567,0.849969,0.281696,0.862826,0.270691,0.872812,9.966270,2.602810,0.008130,0.009350
2,dataset_graph_cooccurrence_1_all_cade-ana,307.748487,92.900485,0.243561,0.938014,"PassiveAggressiveClassifier(C=1.0, average=Fal...",balanced,1000,0.001,NaN,...,0.258923,0.946991,0.241149,0.929337,0.230602,0.937713,11.610318,8.674349,0.011687,0.007210
5,dataset_graph_cooccurrence_1_all_cade-ana,302.817280,102.061950,0.209691,0.875408,"PassiveAggressiveClassifier(C=1.0, average=Fal...",balanced,1000,0.001,NaN,...,0.218527,0.876140,0.208559,0.876317,0.201980,0.873767,40.261696,38.944134,0.006803,0.001162
4,dataset_graph_cooccurrence_1_all_cade-ana,311.199031,90.084631,0.184661,0.934641,"PassiveAggressiveClassifier(C=1.0, average=Fal...",balanced,1000,0.001,NaN,...,0.195836,0.944906,0.182144,0.930361,0.175997,0.928655,8.408203,4.713523,0.008293,0.007292
0,dataset_graph_cooccurrence_1_all_ling-spam,167.608820,14.716106,0.985563,1.000000,"PassiveAggressiveClassifier(C=1.0, average=Fal...",balanced,1000,0.001,NaN,...,0.984863,1.000000,0.981171,1.000000,0.990656,1.000000,4.483722,2.948857,0.003903,0.000000
1,dataset_graph_cooccurrence_1_all_ling-spam,88.435672,16.592627,0.984523,0.992894,"PassiveAggressiveClassifier(C=1.0, average=Fal...",balanced,1000,0.001,NaN,...,0.985164,0.987987,0.981359,0.996272,0.987047,0.994422,58.157365,1.952002,0.002366,0.003551
2,dataset_graph_cooccurrence_1_all_ling-spam,50.403759,17.296075,0.907569,1.000000,"PassiveAggressiveClassifier(C=1.0, average=Fal...",balanced,1000,0.001,NaN,...,0.894038,1.000000,0.902487,1.000000,0.926195,1.000000,0.400361,4.871233,0.013611,0.000000
4,dataset_graph_cooccurrence_1_all_ling-spam,39.117641,14.126546,0.820692,1.000000,"PassiveAggressiveClassifier(C=1.0, average=Fal...",balanced,1000,0.001,NaN,...,0.828987,1.000000,0.817830,1.000000,0.815252,1.000000,0.902149,2.904087,0.005962,0.000000
